In [37]:
import pyautogui as gui
import pywinauto as wind
import pytesseract
from time import sleep
from pywinauto.application import Application
import path_location

pytesseract.pytesseract.tesseract_cmd = path_location.TESSERACT_LOC
RST_LOCATION = path_location.RST_LOCATION

In [38]:
def click_here(x, y, pause=None, debug=False):
    gui.moveTo(x, y, pause=pause)
    if debug:
        gui.press('ctrl', presses=2, interval=.7)
    gui.click()

def get_app_prop(app):
    app_prop = app.rectangle()
    return (app_prop.left, app_prop.top, app_prop.width(), app_prop.height())

def get_sreenshot(app=None, wind_stat=None):
    if app != None:
        app.set_focus()
        app_prop = get_app_prop(app)
        wind_stat = (app_prop[0], app_prop[1], app_prop[2], app_prop[3])
    elif wind_stat == None:
        print('[ERROR] Either app object or custom box has to be passed')
        return 
    return gui.screenshot(region=wind_stat)

def image_convert(image):
    greyscale = image.convert('L')
    blackwhite = greyscale.point(lambda x: 0 if x < 200 else 255, '1')
    return image
    # return blackwhite

In [39]:
def ocr(img, v1=False, raw=False):
    if v1:
        return pytesseract.image_to_string(img).split('\n')
    else:
        import io
        import json
        import cv2
        import numpy as np
        import requests

        img.save("screenshot.jpg")
        img = cv2.imread("screenshot.jpg")
        url_api = "https://api.ocr.space/parse/image"
        _, compressedimage = cv2.imencode(".jpg", img, [1, 90])
        file_bytes = io.BytesIO(compressedimage)

        result = requests.post(url_api,
                    files = {"screenshot.jpg": file_bytes},
                    data = {"apikey": path_location.OCR_SPACE_API_KEY,
                            "language": "eng",'isOverlayRequired':raw})
        result = result.content.decode()
        result = json.loads(result)
        if raw:
            return result
        parsed_results = result.get("ParsedResults")[0]
        return parsed_results.get("ParsedText").split()

In [42]:
try:
    pc2 = Application().connect(title_re='.*Project CARS 2.*')['Project Cars 2']
except wind.findwindows.ElementNotFoundError:
    print('[ERROR] Launch PCARS 2 first')
#     quit()
    
try:
    rst = Application().connect(title_re='MainWindow').MainWindow
except wind.findwindows.ElementNotFoundError:
    rst = Application().start(RST_LOCATION+'\\RST Software.exe').MainWindow
except wind.findwindows.ElementAmbiguousError:
    print('[ERROR] Sad, but for now, close the params child window.')
#     quit()

D:\Users\GV\Anaconda3\lib\site-packages\pywinauto\application.py:1087: UserWarning: 32-bit application should be automated using 32-bit Python (you use 64-bit Python)
  UserWarning)


## RST

In [43]:
def open_rst_params():
    rst.set_focus()
    RST_FILE_OFFSET_X = 370
    RST_FILE_OFFSET_Y = 30
    RST_NEXT_OFFSET_X = RST_FILE_OFFSET_X+10
    RST_NEXT_OFFSET_Y = RST_FILE_OFFSET_Y+80

    click_here(x=get_app_prop(rst)[0]+RST_FILE_OFFSET_X,y=get_app_prop(rst)[1]+RST_FILE_OFFSET_Y, pause=1)
    click_here(x=get_app_prop(rst)[0]+RST_NEXT_OFFSET_X,y=get_app_prop(rst)[1]+RST_NEXT_OFFSET_Y, pause=1)

def find_rst_settings(pos):
    next_sess = get_app_prop(rst)
    mid_op = next_sess[2]/4
    return (next_sess[0]+mid_op/2+mid_op*pos, next_sess[1]+100)

def open_rst_sus():
    rst.set_focus()
    next_sess_sus = find_rst_settings(1)
    clic_here(next_sess_sus, pause=1, debug=True)

def open_rst_dam():
    rst.set_focus()
    next_sess_sus = find_rst_settings(2)
    click_here(next_sess_sus, pause=1, debug=True)

def open_rst_gear():
    rst.set_focus()
    next_sess_gear = find_rst_settings(3)
    click_here(next_sess_gear, pause=1, debug=True)

open_rst_params()

## PCARS2

In [47]:
WIDTH_RATIO = pc2.client_rect().width()/2560
HEIGHT_RATIO = pc2.client_rect().height()/1440
PC2_SETUP_HEAD_CENTER = int(pc2.rectangle().left+pc2.rectangle().width()/2)
PC2_SETUP_HEAD_Y = int(get_app_prop(pc2)[1]+360*HEIGHT_RATIO)
PC2_POS_SUS_X = int(PC2_SETUP_HEAD_CENTER-200)
PC2_POS_DAMP_X = int(PC2_SETUP_HEAD_CENTER)
PC2_POS_ECU_X = int(PC2_SETUP_HEAD_CENTER+250)


In [192]:
def app_draw_outline(app, x, y, height, width, use_ratio=False):
    # PCARS 2 options change based on windows size (resolution)
    if use_ratio:
        app_h = app.client_rect().height()
        app_w = app.client_rect().width()
        x = int(x * app_w/2560)
        y = int(y * app_h/1440)
        width = int(width * app_w/2560)
        height = int(height * app_h/1440)

    m = app.client_to_screen((x,y))
    class cusrect:
        left=m[0]
        top=m[1]
        right=left+width
        bottom=top+height


    for _ in range(50):
        app.draw_outline(rect=cusrect)

In [193]:
app_draw_outline(pc2, x=865, y=590, height=170, width=130, use_ratio=False)

865 590 170 130


In [33]:
def get_stats_gear_pc2(PC2_POS_ECU_X, PC2_SETUP_HEAD_Y, ratio):
    # Select ECU menu
    rst.set_focus()
    pc2.set_focus()
    gui.click(PC2_POS_ECU_X, PC2_SETUP_HEAD_Y, clicks=2,interval=1)
    takestart_x = pc2.rectangle().left+1360*ratio[1]
    takeend_width = 100*ratio[1]
    takestart_y = pc2.rectangle().top+740*ratio[1]
    takeend_height = 320*ratio[1]
    content = image_convert(get_sreenshot(wind_stat=(takestart_x,takestart_y, takeend_width, takeend_height)))
    return ocr(content)

def set_stats_gear_rst(gears):
    rst.set_focus()
    rst_pos_x, rst_pos_y = rst.rectangle().left, rst.rectangle().top
    rst_pos_height, rst_pos_width = rst.rectangle().height(), rst.rectangle().width()
    rst_width_mid = rst_pos_x+rst_pos_width/2
    rst_y_firstfield = rst_pos_y+180
    for gear, val in enumerate(gears):
        gui.click((rst_width_mid, rst_y_firstfield+55*gear))
        try:
            float(val)
            gui.typewrite(val)
        except ValueError:
            print(f'[ERROR] Gear {gear} ({val}) not a number')

In [32]:
def get_stats_sus_pc2(PC2_POS_SUS_X, PC2_SETUP_HEAD_Y, ratio):
    # Select suspension menu
    rst.set_focus()
    pc2.set_focus()
    gui.click(PC2_POS_SUS_X, PC2_SETUP_HEAD_Y, clicks=2,interval=1)

    # Get front suspension stats
    takestart_x = pc2.rectangle().left+1360*ratio[1]
    return get_sreenshot(wind_stat=(takestart_x,takestart_y, takeend_width, takeend_height)))
    # takestart_x = pc2.rectangle().left+1360*ratio[1]
    # takeend_width = 100*ratio[1]
    # takestart_y = pc2.rectangle().top+740*ratio[1]
    # takeend_height = 310*ratio[1]
    # content = image_convert(get_sreenshot(wind_stat=(takestart_x,takestart_y, takeend_width, takeend_height)))
    # return ocr(content)
def set_stats_sus_rst(gears):
    rst.set_focus()
    # rst_pos_x, rst_pos_y = rst.rectangle().left, rst.rectangle().top
    # rst_pos_height, rst_pos_width = rst.rectangle().height(), rst.rectangle().width()
    # rst_width_mid = rst_pos_x+rst_pos_width/2
    # rst_y_firstfield = rst_pos_y+180
    # for gear, val in enumerate(gears):
    #     gui.click((rst_width_mid, rst_y_firstfield+55*gear))
    #     try:
    #         float(val)
    #         gui.typewrite(val)
    #     except ValueError:
    #         print(f'[ERROR] Gear {gear} ({val}) not a number')

get_stats_sus_pc2(PC2_POS_SUS_X, PC2_SETUP_HEAD_Y, (HEIGHT_RATIO, WIDTH_RATIO))

(0.6888888888888889, 0.690625)
1719.25
69.0625
916.0625
221.0


In [0]:
def get_damp_stats(PC2_POS_DAMP_X, PC2_SETUP_HEAD_Y, ratio=None):
    #Select dampers menu
    rst.set_focus()
    pc2.set_focus()
    gui.click(PC2_POS_DAMP_X, PC2_SETUP_HEAD_Y, clicks=2,interval=1)

In [0]:
# print(get_suspension_stats(PC2_POS_SUS_X, PC2_SETUP_HEAD_Y))
# print(get_damp_stats(PC2_POS_DAMP_X, PC2_SETUP_HEAD_Y))

# print(PC2_POS_SUS_X, PC2_SETUP_HEAD_Y)
# print(PC2_POS_DAMP_X, PC2_SETUP_HEAD_Y)
# print(PC2_POS_ECU_X, PC2_SETUP_HEAD_Y)

In [0]:
open_rst_gear()
gears = get_stats_gear_pc2(PC2_POS_ECU_X, PC2_SETUP_HEAD_Y, (HEIGHT_RATIO, WIDTH_RATIO))
set_stats_gear_rst(gears)

In [0]:
content = image_convert(get_sreenshot(pc2))
ocr(content, raw=True)

In [0]:
content